# HIDVL metadata spreadsheet merge script
As part of the process to generate draft HIDVL MARC records by batch, we need to combine metadata spreadsheets from two different sources: the legacy metadata submission form (the data from which is converted via another script from a text file to a CSV file) and the new Airtable metadata submission form.

In [1]:
#import modules and libraries
import pandas as pd
import numpy as np
from datetime import datetime, date, time
filetime = datetime.now()
filetime = filetime.strftime("%Y-%m-%d_%I-%M_%p")

## Legacy metadata
In the first part of this process, we'll deal with the legacy metadata that dates prior to 2019.

In [2]:
pre_2019_dmd = input("enter file name and if appropriate filepath of legacy metadata csv: ")

enter file name and if appropriate filepath of legacy metadata csv:  /Users/alexandra/Downloads/hidvl_dmd_parsed_combined_2019-11-22_05-09_PM_March2022_pre-2019.csv


In [3]:
# load old metadata dataframe from csv
df_pre_2019_dmd = pd.read_csv(pre_2019_dmd,na_filter=False,quotechar = '"')
#print(df_pre_2019_dmd)

In [4]:
#regex replaces whitespace at the end of the cells with NaN; this also has the effect of flipping "blank" cells to NaN
#I'm sure there is a more elegant way to do this.
df_pre_2019_dmd = df_pre_2019_dmd.replace(r'^\s*$', np.nan, regex=True)

In [5]:
#specify new column names for the incoming metadata column headers
pre_2019_dmd_newcols = {
    "HI_number" : "HI",
    "﻿NOID": "NOID",
    "Cataloged Status" : "Cataloged_Status",
    "Publication Cycle" : "Publication_Cycle",
    "Correction_note" : "Correction_Note",
    "Mastering Offset Timecode": "Mastering_Offset_Timecode",
    "Language": "Language_Note",
    "Subjects" : "Subjects_653",
    "Participants" : "Participants_old"

}

In [6]:
#rename the column headers
df_pre_2019_dmd.rename(columns=pre_2019_dmd_newcols, inplace=True)
#print("new df",df_pre_2019_dmd)

In [7]:
#add an empty column for 650 subjects
#based on https://stackoverflow.com/questions/16327055/how-to-add-an-empty-column-to-a-dataframe
df_pre_2019_dmd["Subjects_650"] = np.nan

In [8]:
#strip the period from the end of the production credits and participants fields
#based on https://stackoverflow.com/questions/37001787/remove-ends-of-string-entries-in-pandas-dataframe-column
df_pre_2019_dmd["Additional_Production_Credits"] = df_pre_2019_dmd["Additional_Production_Credits"].str.rstrip('.')
df_pre_2019_dmd["Participants_old"] = df_pre_2019_dmd["Participants_old"].str.rstrip('.')

In [9]:
#concatenate all of the participant, additional production credits, and performer columns.
#the catch is that sometimes these columns are empty!
#based on https://stackoverflow.com/questions/60724940/concatenate-strings-across-columns-that-are-not-null
df_pre_2019_dmd["Participants"] = df_pre_2019_dmd[["Additional_Production_Credits", "Participants_old", "Performers"]].apply(lambda x: '; '.join(x.dropna()), axis=1)
#print the participants field
print(df_pre_2019_dmd["Participants"])
#print a sample row to check that concatenation worked
print(df_pre_2019_dmd.loc[3,'Participants'])

0    Roland Marconi, editor; Melody Bates, Marco Ce...
1    Roland Marconi, editor; Melody Bates, Marco Ce...
2    Roland Marconi, editor; Melody Bates, Marco Ce...
Name: Participants, dtype: object


KeyError: 3

In [10]:
#get rid of the extra columns that we just concatenated into the new participants field
df_pre_2019_dmd.drop(["Additional_Production_Credits","Participants_old","Performers"], axis=1, inplace=True)
#get some info about the dataframe
print(df_pre_2019_dmd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   HI                         3 non-null      object 
 1   NOID                       3 non-null      object 
 2   Cataloged_Status           0 non-null      object 
 3   Publication_Cycle          0 non-null      object 
 4   Correction_Note            3 non-null      object 
 5   Format                     3 non-null      object 
 6   Source_Tape_Generation     3 non-null      object 
 7   Mastering_Offset_Timecode  0 non-null      object 
 8   Run_Time                   3 non-null      object 
 9   Series_Title               3 non-null      object 
 10  Meeting_Information        0 non-null      object 
 11  Title                      3 non-null      object 
 12  Alternate_Titles           3 non-null      object 
 13  Date_of_Production         3 non-null      object 
 14

## Airtable metadata
In the second part of this process, we'll deal with the new metadata that dates from 2020 to the present.

In [11]:
post_2019_dmd = input("enter file name and if appropriate filepath of airtable metadata csv: ")

enter file name and if appropriate filepath of airtable metadata csv:  /Users/alexandra/Downloads/Metadata-March_2022.csv


In [12]:
# load new metadata dataframe from csv
df_post_2019_dmd = pd.read_csv(post_2019_dmd,na_filter=False,quotechar = '"')

In [13]:
df_post_2019_dmd = df_post_2019_dmd.replace(r'^\s*$', np.nan, regex=True)

In [14]:
df_post_2019_dmd = df_post_2019_dmd.fillna(np.nan)

In [15]:
# specify new column names for the incoming metadata column headers
post_2019_dmd_newcols = {
    "HI #" : "HI",
    "Inventory": "NOID",
    "Publication cycle" : "Publication_Cycle",
    "Date of event" : "Date_of_Production",
    "Location information": "Location_Venue",
    "Language note": "Language_Note",
    "Language": "Language_List",
    "Main production credits": "Main_Production_Credits",
    "Event type" : "Worktypes",
    "Subject": "Subjects_653",
    "Copyright holder": "Rights_Holder",
    "Artist bio": "Artist_Bio",
    "Run time rounded":"Run_Time",
    "Collection": "Series_Title",
    "Conference":"Meeting_Information"
}

In [16]:
#rename the column headers
df_post_2019_dmd.rename(columns=post_2019_dmd_newcols, inplace=True)
#see what columns we have in the dataframe now:
#print("new df",df_post_2019_dmd)
print("new df",df_post_2019_dmd.info())
#Alternate titles have imported as non-null float64 values, and I'm not sure why!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   HI                                 27 non-null     object 
 1   NOID                               27 non-null     object 
 2   DMD Finalized                      27 non-null     object 
 3   Title                              27 non-null     object 
 4   Alternate title 1                  1 non-null      object 
 5   Alternate title 2                  0 non-null      float64
 6   Alternate title 3                  0 non-null      float64
 7   Alternate title 4                  0 non-null      float64
 8   Alternate title 5                  0 non-null      float64
 9   Series_Title                       27 non-null     object 
 10  Meeting_Information                1 non-null      object 
 11  Worktypes                          27 non-null     object 
 

In [17]:
#add an empty column for 650 subjects
#based on https://stackoverflow.com/questions/16327055/how-to-add-an-empty-column-to-a-dataframe
df_post_2019_dmd["Subjects_650"] = np.nan

In [18]:
#do some concatenation to populate the format field
df_post_2019_dmd ["Format"] = df_post_2019_dmd["How many source media form items?"].astype(str) + " " + df_post_2019_dmd["Source media format"]

In [19]:
#combine copyright holder contact info into a single field
#based on https://stackoverflow.com/questions/60724940/concatenate-strings-across-columns-that-are-not-null
df_post_2019_dmd["Copyright_Contact"] = df_post_2019_dmd[["Copyright contact designation","Copyright address","Copyright business phone","Copyright mobile phone","Copyright fax","Copyright email 1","Copyright email 2","Copyright email 3","Copyright website"]].apply(lambda x: ', '.join(x.dropna()), axis=1)
df_post_2019_dmd["Copyright_Contact"] = df_post_2019_dmd["Copyright_Contact"].replace('\\n', ', ', regex=True)
print(df_post_2019_dmd["Copyright_Contact"])


0     20 Cooper Square, Fifth Floor, New York, NY 10...
1     20 Cooper Square, Fifth Floor, New York, NY 10...
2     20 Cooper Square, Fifth Floor, New York, NY 10...
3     20 Cooper Square, Fifth Floor, New York, NY 10...
4     20 Cooper Square, Fifth Floor, New York, NY 10...
5     20 Cooper Square, Fifth Floor, New York, NY 10...
6     20 Cooper Square, Fifth Floor, New York, NY 10...
7     20 Cooper Square, Fifth Floor, New York, NY 10...
8     20 Cooper Square, Fifth Floor, New York, NY 10...
9     20 Cooper Square, Fifth Floor, New York, NY 10...
10    Pedro Bennaton, pedrobennaton@errogrupo.com.br...
11    Pedro Bennaton, pedrobennaton@errogrupo.com.br...
12    Pedro Bennaton, pedrobennaton@errogrupo.com.br...
13    Pedro Bennaton, pedrobennaton@errogrupo.com.br...
14    Pedro Bennaton, pedrobennaton@errogrupo.com.br...
15    Pedro Bennaton, pedrobennaton@errogrupo.com.br...
16    Pedro Bennaton, pedrobennaton@errogrupo.com.br...
17    Pedro Bennaton, pedrobennaton@errogrupo.co

In [20]:
#combine alternate titles into a single cell
#based on https://stackoverflow.com/questions/60724940/concatenate-strings-across-columns-that-are-not-null
#this may not actually concatenate any alternate titles, because there is usually only ever one
df_post_2019_dmd["Alternate_Titles"] = df_post_2019_dmd[["Alternate title 1","Alternate title 2","Alternate title 3","Alternate title 4","Alternate title 5"]].apply(lambda x: '|'.join(x.dropna()), axis=1)

In [21]:
#tried filling the blank cells in this field with np.nan but it didn't work...
df_post_2019_dmd["Alternate_Titles"].fillna(np.nan)

0                                                      
1                                                      
2                                                      
3                                                      
4                                                      
5                                                      
6                                                      
7                                                      
8                                                      
9                                                      
10                                                     
11                                                     
12                                                     
13    (ERUPTION: Seedling Depends on the Size of the...
14                                                     
15                                                     
16                                                     
17                                              

In [22]:
#I still wanted to check to see if the values had become null values!
print(df_post_2019_dmd ["Alternate_Titles"].isnull())

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
Name: Alternate_Titles, dtype: bool


In [23]:
#get rid of any newline characters
df_post_2019_dmd["Alternate_Titles"] = df_post_2019_dmd["Alternate_Titles"].replace('\\n', '', regex=True)

In [24]:
#print a sample record that had newlines
print(df_post_2019_dmd.loc[3,"Alternate_Titles"])

In [25]:
#drop unwanted columns and see what remains
df_post_2019_dmd.drop(["Run time","DMD Finalized","How many source media form items?","Source media format","Alternate title 1","Alternate title 2","Alternate title 3","Alternate title 4","Alternate title 5","Copyright contact designation","Copyright address","Copyright business phone","Copyright mobile phone","Copyright fax","Copyright email 1","Copyright email 2","Copyright email 3","Copyright website"], axis=1, inplace=True)

In [26]:
print(df_post_2019_dmd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   HI                       27 non-null     object 
 1   NOID                     27 non-null     object 
 2   Title                    27 non-null     object 
 3   Series_Title             27 non-null     object 
 4   Meeting_Information      1 non-null      object 
 5   Worktypes                27 non-null     object 
 6   Date_of_Production       27 non-null     object 
 7   Location_Venue           27 non-null     object 
 8   Subjects_653             27 non-null     object 
 9   Summary                  27 non-null     object 
 10  Artist_Bio               27 non-null     object 
 11  Rights_Holder            27 non-null     object 
 12  Main_Production_Credits  27 non-null     object 
 13  Participants             27 non-null     object 
 14  Run_Time                 27 

In [27]:
df_combined_dmd = pd.concat([df_pre_2019_dmd,df_post_2019_dmd],ignore_index=True,keys=['pre', 'post'])
#print(df_combined_dmd)
print(df_combined_dmd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 30 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   HI                         30 non-null     object 
 1   NOID                       30 non-null     object 
 2   Cataloged_Status           0 non-null      object 
 3   Publication_Cycle          27 non-null     object 
 4   Correction_Note            3 non-null      object 
 5   Format                     30 non-null     object 
 6   Source_Tape_Generation     3 non-null      object 
 7   Mastering_Offset_Timecode  0 non-null      object 
 8   Run_Time                   30 non-null     object 
 9   Series_Title               30 non-null     object 
 10  Meeting_Information        1 non-null      object 
 11  Title                      30 non-null     object 
 12  Alternate_Titles           30 non-null     object 
 13  Date_of_Production         30 non-null     object 
 

In [28]:
df_combined_dmd = df_combined_dmd[["NOID","Alternate_Titles","Artist_Bio","Copyright_Contact","Date_of_Production","Format","HI","Language_List","Language_Note","Location_Venue","Main_Production_Credits","Meeting_Information","Participants","Publication_Cycle","Rights_Holder","Run_Time","Series_Title","Subjects_650","Subjects_653","Summary","Title","Worktypes"]]

In [29]:
df_combined_dmd.to_csv("hidvl_metadata_combined_%s.csv"%filetime, index=False)

The end!

_Some ideas for improvement: split legacy summary column into summary and bio (currently doing this in OpenRefine)_